In [1]:
import pendulum
lastBusDay = pendulum.now()
from constants import BRKR, DATA, FUTL, SETG, SUPR, UTIL, EMA_SETG
from symbols import Symbols
from omspy_brokers.finvasia import Finvasia
from logzero import logger, logfile
import traceback

def get_ltp(api, symbol_token=None):
    UTIL.slp_til_nxt_sec()
    quote = 0
    try:
        if symbol_token:
            exchange = "NFO"
            symbol_token = str(symbol_token)
        else:
            exchange = SETG[SYMBOL]['key'].split("|")[0]
            symbol_token = SETG[SYMBOL]['key'].split("|")[1]
        resp = api.finvasia.get_quotes(exchange, symbol_token)
        if resp is None:
            raise Exception("No response")
        else:
            quote = int(float(resp["lp"]))
    except Exception as e:
        logger.warning(f"{e} while getting ltp for {symbol_token=}")
        print(traceback.format_exc())
    finally:
        return quote


def get_api():
    brkr = Finvasia(**BRKR)
    if not brkr.authenticate():
        logger.error("Failed to authenticate")
        __import__("sys").exit(0)
    else:
        print("Authenticated")
        return brkr
SYMBOL ="BANKNIFTY"
EXPIRY = SETG[SYMBOL]['expiry']
DIFF = SETG[SYMBOL]['diff']
api = get_api()
O_SYM = Symbols("NFO", SYMBOL, EXPIRY, DIFF)
O_SYM.get_exchange_token_map_finvasia()
dct_symtkns = O_SYM.get_all_tokens_from_csv()
atm = O_SYM.get_atm(get_ltp(api))
ce_option, pe_option = O_SYM.find_itm_option(atm, "C"), O_SYM.find_itm_option(atm, "P")

fromBusDay = lastBusDay.replace(
        hour=9, minute=15, second=0, microsecond=0).subtract(days=7)  
resp = api.historical(
    "NFO", dct_symtkns[ce_option], fromBusDay.timestamp(), lastBusDay.timestamp(), 15
)

looking up c:\Users\dell\miniconda3\Lib\site-packages\omspy_brokers\finvasia.yaml
Authenticated
False: 2024-04-10 == 2024-04-10


In [6]:
fromBusDay = lastBusDay.replace(
        hour=9, minute=15, second=0, microsecond=0).subtract(days=7)  
resp = api.historical(
    "NFO", str(dct_symtkns[ce_option]), fromBusDay.timestamp(), lastBusDay.timestamp(), 15
)

In [7]:
resp

[{'stat': 'Ok',
  'time': '10-04-2024 15:15:00',
  'ssboe': '1712742300',
  'into': '789.50',
  'inth': '796.45',
  'intl': '760.30',
  'intc': '774.60',
  'intvwap': '776.55',
  'intv': '63510',
  'intoi': '-25905',
  'v': '783045',
  'oi': '415065'},
 {'stat': 'Ok',
  'time': '10-04-2024 15:00:00',
  'ssboe': '1712741400',
  'into': '834.30',
  'inth': '845.40',
  'intl': '772.75',
  'intc': '789.90',
  'intvwap': '815.95',
  'intv': '103920',
  'intoi': '-9285',
  'v': '719535',
  'oi': '440970'},
 {'stat': 'Ok',
  'time': '10-04-2024 14:45:00',
  'ssboe': '1712740500',
  'into': '830.65',
  'inth': '838.95',
  'intl': '813.30',
  'intc': '833.25',
  'intvwap': '831.91',
  'intv': '30510',
  'intoi': '-1815',
  'v': '615615',
  'oi': '450255'},
 {'stat': 'Ok',
  'time': '10-04-2024 14:30:00',
  'ssboe': '1712739600',
  'into': '797.35',
  'inth': '850.20',
  'intl': '788.30',
  'intc': '833.65',
  'intvwap': '828.91',
  'intv': '47445',
  'intoi': '-10680',
  'v': '585105',
  'oi': 

In [17]:
import pandas as pd
df = pd.DataFrame(resp).iloc[:100].iloc[::-1]
df = df[["time", "intc"]]
df['timestamp'] = pd.to_datetime(df['time'], format='%d-%m-%Y %H:%M:%S').astype('int64')// 10**9
df.rename(columns={"intc": "close", "time":"timestamp_column"}, inplace=True)
df["close"] = df["close"].astype("int")

,timestamp_column,close,timestamp
99,05-04-2024 09:15:00,452.45,1712308500
98,05-04-2024 09:30:00,437.30,1712309400
97,05-04-2024 09:45:00,444.90,1712310300
96,05-04-2024 10:00:00,431.20,1712311200
95,05-04-2024 10:15:00,458.45,1712312100
...,...,...,...
4,10-04-2024 14:15:00,795.40,1712758500
3,10-04-2024 14:30:00,833.65,1712759400
2,10-04-2024 14:45:00,833.25,1712760300
1,10-04-2024 15:00:00,789.90,1712761200


In [3]:
ce_option, pe_option

('BANKNIFTY24APR24C48500', 'BANKNIFTY24APR24P49500')